In [1]:
#import global packages
import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii

# import local packages
import os, sys
sys.path.append('./Python')
from getMapValue import getValue
from GalactocentricRadius import gc, gcr

#H-alpha attenuation corrected flux extension
ext = "HA_FLUX_CORR"

In [2]:
# Compile galaxy and sne data
data = Table.read('../Data/1.GroupedSampleData.csv', format='csv')

data[0:1]

# read in metallicity gradients
metal = Table.read('../Data/0.groves_scal_gradients.csv', format='csv')
metal[0:1]

# join tables

data = join(data, metal, ["galaxy"])
data[0:1]

col0,galaxy,distance,gal_ra,gal_dec,pos_ang,incl,nat_res,r_eff,maps_musedap,maps_nat,maps_150,maps_1kpc,maps_HII,maps_irac,maps_CO21,maps_eCO21,maps_ext,maps_vdisp,maps_HST,JWST200,JWST300,JWST335,JWST360,JWST770,JWST1000,JWST1130,JWST2100,sn_name,sn_type,sn_ra,sn_dec,Intercept,Intercept_error,Slope,Slope_error,Value,Sigma
int64,str7,float64,float64,float64,float64,float64,float64,float64,str71,str58,str64,str63,str50,str40,str72,str73,str36,str32,str45,str93,str93,str93,str93,str93,str94,str94,str94,str82,str50,str127,str132,float64,float64,float64,float64,float64,float64
0,NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,42.091877,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,../Data/SFR_v1/NGC1087_MAPS_sfr.fits,../Data/vdisp/ngc1087_vdisp.fits,../../GalaxyFiles/HST/ngc1087_hst_ha_sic.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F200W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F300W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F330W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F365W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F770W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1000W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1130W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F2100W_atGauss7p5_anchored.fits,['SN1995V'],['II'],[41.61152777777777],[-0.4987861111111111],8.479,0.0,-0.07,0.01,8.409,0.032


In [3]:
# prepare lists for getValue method

centerCoords = [(float(data["gal_ra"][i]), float(data["gal_dec"][i])) for i in range(len(data))] 
sn_ras, sn_decs, sn_names, sn_types = [],[],[],[]
for i in range(len(data)):
    sn_ras.append(data["sn_ra"][i][1:-1].replace(",","").split(" "))
    sn_decs.append(data["sn_dec"][i][1:-1].replace(",","").split(" "))
    sn_names.append(data["sn_name"][i][1:-1].replace(",","").replace("'","").split(" "))

for i in range(len(sn_ras)):
    sn_ras[i] = [float(ele) for ele in sn_ras[i]]
    sn_decs[i] = [float(ele) for ele in sn_decs[i]]
    
sn_name = np.hstack(sn_names)

sn_type = ["II","II",'Ia', 'II', 'SESNe', 'Unc', "II", "Ia", "Ia", "II", 'II', 
           'SESNe', 'II', 'Ia', 'II', 'II', 'II','II', 'II', 'II', 'SESNe', 'II', 
           'II', 'II', 'II', 'II', 'II', 'Ia', 'Ia', 'Unc', 'Unc', 'SESNe', 'SESNe']

In [4]:
data["sn_type"]

['II']
['II']
"['Ia', 'II', 'Ic', 'Unclassified']"
['II']
"['Ia', 'Ia-02cx', 'II']"
"['II', 'Ib/c']"
"['II', 'Ia', 'II P', 'II', 'IIn']"
"['II', 'II', 'II', 'Ic']"
"['II P', 'II', 'II', 'II', 'II P', 'II', 'Ia Pec']"
"['Ia', 'I', 'I', 'Ic', 'Ib']"


In [5]:
hdu=pyfits.open(data["maps_musedap"][0])
hdu["HA6562_SIGMA"].data

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [6]:
# get intensity and extinction values for each SN

int_nat, int_150, int_1kpc, a_v, c_o, e_co, lw, rad, gcr_reff, met = [],[],[],[],[],[],[],[],[],[]
for i in range(len(sn_ras)):
    if len(sn_ras[i]) < 1:
        int_nat.append(getValue(data["maps_nat"][i], ext, sn_ras[i], sn_decs[i]))
        int_150.append(getValue(data["maps_150"][i], ext, sn_ras[i], sn_decs[i]))
        int_1kpc.append(getValue(data["maps_1kpc"][i], ext, sn_ras[i], sn_decs[i]))
        a_v.append(3.1 * (getValue(data["maps_nat"][i], "EBV", sn_ras[i], sn_decs[i])))
        c_o.append(getValue(data["maps_CO21"][i], 0, sn_ras[i], sn_decs[i]))
        e_co.append(getValue(data["maps_eCO21"][i], 0, sn_ras[i], sn_decs[i]))
        lw.append(getValue(data["maps_musedap"][i], "HA6562_SIGMA", sn_ras[i], sn_decs[i]))
# #         line = getValue(data["maps_vdisp"][i], 0, sn_ras[i], sn_decs[i])
#         if np.isfinite(line):
#             lw.append(str(round(line)))
#         else:
#             lw.append("-")        
        r_deg, r_asec, proj = gc(data["gal_ra"][i], data["gal_dec"][i], data["incl"][i], data["pos_ang"][i], 
                                   sn_ras[i], sn_decs[i]) 
        r = gcr(r_deg, data["distance"][i])
        reff = r / (data["r_eff"][i])
        rad.append(r/1000)
        gcr_reff.append(reff)
        met.append(reff * data["Slope"][i] + data["Intercept"][i])

    else:
        for j in range(len(sn_ras[i])):
            int_nat.append(getValue(data["maps_nat"][i], ext, sn_ras[i][j], sn_decs[i][j]))
            int_150.append(getValue(data["maps_150"][i], ext, sn_ras[i][j], sn_decs[i][j]))
            int_1kpc.append(getValue(data["maps_1kpc"][i], ext, sn_ras[i][j], sn_decs[i][j]))
            a_v.append(3.1 * (getValue(data["maps_nat"][i], "EBV", sn_ras[i][j], sn_decs[i][j])))
            c_o.append(getValue(data["maps_CO21"][i], 0, sn_ras[i][j], sn_decs[i][j]))
            e_co.append(getValue(data["maps_eCO21"][i], 0, sn_ras[i][j], sn_decs[i][j]))
            lw.append(getValue(data["maps_musedap"][i], "HA6562_SIGMA", sn_ras[i][j], sn_decs[i][j]))
#             line = getValue(data["maps_vdisp"][i], 0, sn_ras[i][j], sn_decs[i][j])
#             if np.isfinite(line):
#                 lw.append(str(round(line)))
#             else:
#                 lw.append("-")
            r_deg, r_asec, proj = gc(data["gal_ra"][i], data["gal_dec"][i], data["incl"][i], data["pos_ang"][i], 
                                       sn_ras[i][j], sn_decs[i][j]) 
            r = gcr(r_deg, data["distance"][i])
            reff = r / (data["r_eff"][i])
            rad.append(r/1000)
            gcr_reff.append(reff)
            met.append(reff * data["Slope"][i] + data["Intercept"][i])
            
met[2] = 8.611878787878789
met[3] = 8.528133333333333
met[4] = 8.581115151515151
met[5] = 8.524145454545454 



print("Native Intensities")
print("-------------------")
print(int_nat)
print()
print("150pc Intensities")
print("-------------------")
print(int_150)
print()
print("1kpc Intensities")
print("-------------------")
print(int_1kpc)
print()
print("A_V")
print("-------------------")
print(a_v)
print()
print("CO_21")
print("-------------------")
print(c_o)
print()
print("eCO_21")
print("-------------------")
print(e_co)
print()
print("lw")
print("-------------------")
print(lw)
print()
print("gcr [kpc]")
print("-------------------")
print(rad)
print()
print("gcr [reff]")
print("-------------------")
print(gcr_reff)
print()
print("met")
print("-------------------")
print(met)
print()

Native Intensities
-------------------
[8.839672735197591e+39, 3.445745101042276e+38, 8.904660571183651e+37, 1.3530435354080875e+39, 1.4845660990817182e+40, 2.938518960212585e+39, 2.4790371281538853e+38, 1.263977781860364e+38, 3.8272402829093126e+38, 6.206906305070956e+39, 9.694817596411935e+40, 4.185157273316843e+39, 1.1176985848142032e+40, 1.2565095465358724e+40, 5.177867242845618e+39, 3.0814818947082474e+40, 9.927623759273137e+38, 1.4346514342074687e+40, 3.6180321930942297e+39, 5.1524628176390714e+39, 3.366146687872796e+40, 1.2852723829328137e+40, 8.237240338373292e+38, 4.150635721869112e+39, 2.9303903692530505e+38, 2.199480578104416e+39, 3.314915130256522e+39, 8.951255433113478e+38, 3.664262269436105e+38, 3.606657809150872e+39, 2.1120944464704693e+39, 5.473226299514552e+40, 1.0400343563277594e+39]

150pc Intensities
-------------------
[9.098356574390131e+39, 5.543860583782077e+38, 9.61137910255348e+37, 1.8635576999760546e+39, 1.2147806235754843e+40, 3.591077078009263e+39, 2.553918

In [7]:
# flag whether CO or not at SN sites
co_snr = [x/y for x,y in zip(c_o,e_co)]
co_snr = np.clip(co_snr, 0, np.max(co_snr))

co = []
for i in range(len(co_snr)):
    if co_snr[i] >= 3.:
        co.append("Yes")
    else:
        co.append("No")

In [8]:
intensities = Table({"Supernova" : sn_name,
                    "sn_type"   : sn_type,
                    "int_nat"    : int_nat,
                    "int_150"    : int_150,
                    "int_1kpc"   : int_1kpc,
                    "a_v"        : a_v,
                    "c_o"        : c_o,
                    "e_co"       : e_co,
                    "co_snr"     : co_snr,
                    "co"         : co,
                    "halpha_lws" : lw,
                    "gcr"        : rad,
                    "gcr_reff"   : gcr_reff,
                    "met_theory" : met})

intensities.write("../Data/1.sn_ints_exts.csv", format = "csv", overwrite = True)

In [9]:
intensities

Supernova,sn_type,int_nat,int_150,int_1kpc,a_v,c_o,e_co,co_snr,co,halpha_lws,gcr,gcr_reff,met_theory
str11,str5,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64
SN1995V,II,8.839672735197591e+39,9.098356574390131e+39,5.626363456901733e+39,1.2788075992834158,10.1257905960083,0.9063823223114014,11.17165470123291,Yes,52.49597118323195,0.14295581272079702,3.396280301797827,8.241260378874152
SN2022acko,II,3.445745101042276e+38,5.543860583782077e+38,7.868984950864679e+38,0.3049447396212661,-0.13982966542243958,0.7117888331413269,0.0,No,66.20645202360187,0.3791372256557472,5.349132803990211,8.194418508484775
SN2012fr,Ia,8.904660571183651e+37,9.61137910255348e+37,2.866850419201407e+38,0.0,-2.3060925006866455,0.9818917512893677,0.0,No,67.80803058745335,0.31117160065160954,16.537606327147618,8.611878787878789
SN2001du,II,1.3530435354080875e+39,1.8635576999760546e+39,4.641739419198077e+39,0.17916425772905134,2.8034262657165527,1.1442593336105347,2.4499919414520264,No,67.5783735181326,0.7901610432754345,41.99410306523355,8.528133333333333
SN1983V,SESNe,1.4845660990817182e+40,1.2147806235754843e+40,1.5745029644344602e+39,0.9452981371543961,5.302032947540283,0.9777513742446899,5.422679901123047,Yes,65.62497233234417,0.4849807653394629,25.77491312390853,8.581115151515151
SN1957C,Unc,2.938518960212585e+39,3.591077078009263e+39,4.5113659318499504e+39,0.3792970400406018,nan,nan,nan,No,63.37777764994122,0.8122435748346881,43.167706995891166,8.524145454545454
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN2006ov,II,3.314915130256522e+39,5.0060971152955296e+39,9.40643826722596e+39,0.32560772737337573,-0.7098835706710815,1.3054817914962769,0.0,No,68.26887004833223,0.257649507813319,6.192207244627599,8.414849368171916
SN2014dt,Ia,8.951255433113478e+38,1.2104713775083824e+39,5.085551358668022e+39,0.27866560361769,1.2786945104599,0.9955321550369263,1.284433126449585,No,71.26664845774208,0.19496123908766105,4.685591706912551,8.463061065378797
